Algoritmos Baseados em Correlacionamento

In [ ]:
# Importar a função para upload de arquivos no Colab
from google.colab import files

# Fazer o upload do vídeo manualmente
uploaded = files.upload()

# Importar bibliotecas
import cv2
import matplotlib.pyplot as plt

# Função para exibir imagens no Colab
def exibir_imagem(frame):
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

# Pegar o nome do vídeo carregado
video_path = list(uploaded.keys())[0]
cap = cv2.VideoCapture(video_path)

# Ler o primeiro quadro do vídeo
ret, frame = cap.read()

# Verificar se o vídeo foi carregado corretamente
if not ret:
    print("Erro ao carregar o vídeo.")
else:
    # Definir uma ROI manualmente (ajuste o tamanho para cobrir mais o objeto)
    h, w, _ = frame.shape
    roi = (int(w/2 - 70), int(h/2 - 70), 140, 140)  # ROI ajustada de 140x140

    # Exibir o primeiro quadro com a ROI
    p1 = (int(roi[0]), int(roi[1]))
    p2 = (int(roi[0] + roi[2]), int(roi[1] + roi[3]))
    cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
    exibir_imagem(frame)

    # Inicializar o rastreador CSRT (mais robusto)
    tracker = cv2.TrackerCSRT_create()

    # Inicializar o rastreador com o primeiro quadro e a ROI
    tracker.init(frame, roi)

    # Rastrear o objeto no vídeo
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Atualizar o rastreador
        ok, bbox = tracker.update(frame)

        if ok:
            # Desenhar a caixa delimitadora se o rastreamento for bem-sucedido
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
        else:
            # Exibir mensagem de falha no rastreamento
            cv2.putText(frame, "Falha no rastreamento", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

        # Exibir o quadro atualizado
        exibir_imagem(frame)

    cap.release()

"""Algoritmos Baseados em Regressão"""

# Importar bibliotecas
import cv2
import matplotlib.pyplot as plt

# Função para exibir imagens no Colab
def exibir_imagem(frame):
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

# Carregar vídeo de teste do OpenCV
cap = cv2.VideoCapture("vtest.avi")  # Substitua pelo caminho correto para o vídeo

# Ler o primeiro quadro do vídeo
ret, frame = cap.read()
if not ret:
    print("Erro ao carregar o vídeo.")
else:
    # Definir uma Região de Interesse (ROI) manualmente
    h, w, _ = frame.shape
    roi = (int(w/4), int(h/4), int(w/2), int(h/2))  # ROI maior para rastreamento

    # Exibir o primeiro quadro com a ROI
    p1 = (int(roi[0]), int(roi[1]))
    p2 = (int(roi[0] + roi[2]), int(roi[1] + roi[3]))
    cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
    exibir_imagem(frame)

    # Inicializar o rastreador MOSSE
    tracker = cv2.legacy.TrackerMOSSE_create()

    # Inicializar o rastreador com o primeiro quadro e a ROI
    tracker.init(frame, roi)

    # Rastrear o objeto no vídeo
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Atualizar o rastreador
        ok, bbox = tracker.update(frame)

        if ok:
            # Desenhar a caixa delimitadora se o rastreamento for bem-sucedido
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
        else:
            # Exibir mensagem de falha no rastreamento
            cv2.putText(frame, "Falha no rastreamento", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

        # Exibir o quadro atualizado
        exibir_imagem(frame)

    cap.release()

"""Algoritmos Baseados em Fluxo Óptico



"""

import cv2
import numpy as np
from google.colab.patches import cv2_imshow  # Usar o cv2_imshow no Colab

# Carregar o vídeo
cap = cv2.VideoCapture("vtest.avi")

# Definir parâmetros para o algoritmo de Lucas-Kanade
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Gerar algumas cores aleatórias para desenhar as trilhas
color = np.random.randint(0, 255, (100, 3))

# Ler o primeiro quadro e detectar pontos de interesse
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

# Criar uma máscara de imagem para desenhar as trilhas
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calcular o fluxo óptico
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Selecionar os bons pontos
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Desenhar as trilhas
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

    img = cv2.add(frame, mask)

    # Mostrar o quadro com as trilhas usando cv2_imshow
    cv2_imshow(img)

    # Atualizar os parâmetros para o próximo quadro
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

"""Desempenho entre Diferentes Algoritmos


"""

import cv2
import time

# Carregar o vídeo
video_path = 'vtest.avi'

# Função para medir o desempenho de rastreamento e calcular falhas
def rastrear_objeto(tracker_type):
    # Inicializar rastreadores de acordo com o tipo
    if tracker_type == 'MOSSE':
        tracker = cv2.legacy.TrackerMOSSE_create()
    elif tracker_type == 'CSRT':
        tracker = cv2.TrackerCSRT_create()
    elif tracker_type == 'KCF':
        tracker = cv2.TrackerKCF_create()

    # Ler o vídeo novamente
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if not ret:
        print(f"Erro ao carregar o vídeo com {tracker_type}")
        return

    # Definir uma ROI (Região de Interesse) manual
    h, w, _ = frame.shape
    roi = (int(w/4), int(h/4), int(w/2), int(h/2))  # ROI maior

    # Inicializar o rastreador com o primeiro quadro e a ROI
    tracker.init(frame, roi)

    # Medir o tempo de rastreamento e contar falhas
    tempo_inicial = time.time()
    falhas = 0
    quadros_totais = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        quadros_totais += 1

        # Atualizar o rastreador
        ok, bbox = tracker.update(frame)

        if not ok:
            falhas += 1

    tempo_final = time.time()
    duracao = tempo_final - tempo_inicial

    # Calcular a eficácia (percentual de rastreamento bem-sucedido)
    eficacia = ((quadros_totais - falhas) / quadros_totais) * 100

    # Exibir os resultados
    print(f"\nAlgoritmo: {tracker_type}")
    print(f"Tempo total: {duracao:.2f} segundos")
    print(f"Falhas de rastreamento: {falhas}")
    print(f"Eficácia de rastreamento: {eficacia:.2f}%\n")

# Executar e comparar os três algoritmos
for tracker_type in ['MOSSE', 'CSRT', 'KCF']:
    print(f"\nExecutando rastreamento com {tracker_type}...")
    rastrear_objeto(tracker_type)